In [1]:
from typing import (
    List,
    Tuple,
    NamedTuple
)
import pprint

import pandas as pd

import conftest
import libnano.ensemblrest as er

In [2]:
SPECIES: str = 'human'

In [3]:
GeneInfo = NamedTuple('GeneInfo', [
    ('symbol', str),
    ('barcode', str),
    ('gene_id', str),
    ('transcript_id', str), # canonical id
    ('utr_id', str)  #
    ]
)

In [4]:
GENES_AND_BARCODES: List[GeneInfo] = [
    GeneInfo('SLC17A8', 'ACAGC', 'ENSG00000179520', 'ENST00000323346', 'ENSE00001244923'),
    GeneInfo('GFAP',    'TACAT', 'ENSG00000131095', 'ENST00000638281', 'ENSE00003806990'),
    GeneInfo('FOXO1',   'TTTGC', 'ENSG00000150907', 'ENST00000379561', 'ENSE00001481591'),
    GeneInfo('PSEN2',   'CATTA', 'ENSG00000143801', 'ENST00000366783', 'ENSE00001380688'),
    GeneInfo('DAXX',    'AACCG', 'ENSG00000204209', 'ENST00000374542', 'ENSE00001815438'),
    GeneInfo('CDK5R1',  'CGAGA', 'ENSG00000176749', 'ENST00000313401', 'ENSE00001271015')
]

In [5]:
out_columns: List[str] = [
'symbol',
'gene_id',
'exon_id',
'probe_name',
'probe_seq',
'probe_start',
'probe_end',
'probe_strand',
'probe_length',
'barcode',
'array_freq'
]

# NOTE

Get all of the probe sequences in terms of the transcript RNA strand direction.

Therefore, if the transcript is forward and the probe is reverse, we find the probe and RC it. `er.filterRegionSequence` accomplishes this  

In [6]:
import time
df_out: pd.DataFrame = pd.DataFrame(columns=out_columns)
todfdict = lambda x: {a: b for a, b in zip(out_columns, x)}
NUMBER_PROBES_PER_GENE: int = 0 #10
SLEEP_TIME: float = 0.1
PROBE_CUTOFF_LENGTH: int = 36
for item in GENES_AND_BARCODES:
    three_p_exon_id: str = item.utr_id
    transcript_id: str = item.transcript_id
    filtered_probes: pd.DataFrame = er.getProbesForID(three_p_exon_id, keep_n=NUMBER_PROBES_PER_GENE)
    barcode: str = item.barcode
    exon_seq: str = er.getSequence(three_p_exon_id)
    transcript: dict = er.lookUpID(transcript_id)
    
    for exon in transcript['Exon']:
        if exon['id'] == three_p_exon_id:
            exon_start_idx: int  = exon['start']
            exon_end_idx: int  = exon['end']
    
    for i in range(len(filtered_probes)):
        probe = filtered_probes.iloc[i]
        p_start: int =  probe['start']
        p_end: int =    probe['end']
        p_strand: int = probe['strand']
        p_length: int = probe['probe_length']

        # sometimes p_length doesn't match start and end indices so let's filter those out
        if (p_end - p_start + 1) > p_length:
            print(probe['probe_name'])
            continue
        '''NOTE ADD CODE HERE IF YOU WANT PROBES OF ALL THE SAME LENGTH
        '''
        elif p_length < PROBE_CUTOFF_LENGTH:
            # Try extending from the 5' end of the sequences
            delta: int = PROBE_CUTOFF_LENGTH - p_length
            if p_strand == 1:
                if (p_start > exon_start_idx and 
                    (p_start - exon_start_idx) > delta):
                    p_start -= delta
                else:
                    continue
            else:
                if (p_start < exon_start_idx and 
                    (exon_start_idx - p_start) > delta):
                    p_start += delta # increase the index
                else:
                    continue

        try:
            seq: str = er.getRegionSequence( 
                SPECIES,
                chromosome=probe['seq_region_name'],
                start_idx=p_start,
                end_idx=p_end,
                strand=p_strand
            )
        except:
            print(item.symbol, transcript_id, probe['probe_name'])
            print(p_start, p_end)
            raise
        try:
            was_rc: bool
            seq, was_rc = er.filterRegionSequence(
                    seq,
                    p_strand,
                    transcript_id,
                    transcript,
                    exon_seq
            )
        except ValueError as ex:
            # something was wrong, Region sequence not in transcript_id
            print(ex)
            print(p_start, exon_start_idx)
            print(p_end, exon_end_idx)
            # raise
            continue # skip this probe
        row: list = [
            item.symbol,
            item.gene_id,
            item.utr_id,
            probe['probe_name'],
            seq,
            p_start,
            p_end,
            p_strand,
            probe['probe_length'],
            barcode,
            probe['array_freq']
        ]
        df_out = df_out.append(todfdict(row), ignore_index=True)
        time.sleep(SLEEP_TIME)  # sleep such that requests don't time out
    # end for
    time.sleep(SLEEP_TIME)

Region sequence not in transcript_id: ENST00000374542: -1, rc: False
Region sequence not in transcript_id: ENST00000313401: 1, rc: False


In [7]:
df_out

,symbol,gene_id,exon_id,probe_name,probe_seq,probe_start,probe_end,probe_strand,probe_length,barcode,array_freq
0,SLC17A8,ENSG00000179520,ENSE00001244923,65:723;,GTTCCTCATAGCTGCCCTGGTGCATTACAGTGGTGT,100419838,100419873,1,25,ACAGC,1
1,SLC17A8,ENSG00000179520,ENSE00001244923,307:189;,GATAGAACTCAACCATGAGAGTTTTGCGAGTCCCAA,100419982,100420017,1,25,ACAGC,1
2,SLC17A8,ENSG00000179520,ENSE00001244923,589:220;,GAGTCCCAAAAAGAAGATGTCTTATGGAGCCACCTC,100420009,100420044,1,25,ACAGC,1
3,SLC17A8,ENSG00000179520,ENSE00001244923,GE533132,AGAAGAAGGAATGGAAAGGACAGAGAGGAGCGACCC,100420062,100420097,1,30,ACAGC,1
4,SLC17A8,ENSG00000179520,ENSE00001244923,A_14_P106429,TTCTCAACTATATCCTAATGTCTGAGAGGCACTTCTGTCTTCTCCT...,100420142,100420201,1,60,ACAGC,1
5,SLC17A8,ENSG00000179520,ENSE00001244923,103:636;,ACTATATCCTAATGTCTGAGAGGCACTTCTGTCTTC,100420148,100420183,1,25,ACAGC,1
6,SLC17A8,ENSG00000179520,ENSE00001244923,674:405;,TCTGAGAGGCACTTCTGTCTTCTCCTTACTTTAGAA,100420162,100420197,1,25,ACAGC,1
7,SLC17A8,ENSG00000179520,ENSE00001244923,420:510;,TTAGAAACAGAAAGTATCCATACCTATTGCCTTTCT,100420192,100420227,1,25,ACAGC,1
8,SLC17A8,ENSG00000179520,ENSE00001244923,217:234;,GTATCCATACCTATTGCCTTTCTTGTAGCCCAGCTT,100420205,100420240,1,25,ACAGC,1
9,SLC17A8,ENSG00000179520,ENSE00001244923,cg12083535,CGCTCAGTTGATAACATAGTTGATAATACATATTTTTTGAATTGAC...,100420336,100420385,-1,50,ACAGC,1


# Padlock structure reminder
left and right are in terms of the hybridized sequence

LINEAR VERSION:

    5'    Right Arm       Scaffold Seq (aka Loop)      Left Arm     3'
    +------------------>+-----------~-----------++------------------>

    HYBRIDIZED VERSION

                     Scaffold Seq (aka Loop)
            -------------------~--------------------
            |                                      |
            <     Left Arm    3' 5'   Right Arm    +
    3'      +------------------>+------------------>     5'
    <----------------------------------------------------+
              copied RT'd cDNA reverse strand
              
The scaffold seq looks like this:

    5'+--TTCCTTT-[barcode_solid]---[T2S_sequence]--TCTT->3'

We will use 5 mer barcodes

In [8]:
T2S_SEQ: str =    'ACTTCAGCTGCCCCGGGTGAAGA'
RIGHT_LOOP: str = 'TTCCTTT'
LEFT_LOOP: str =  'TCTT'

In [9]:
len(T2S_SEQ)

23

the padlocks will be:

In [10]:
df_padlocks: pd.DataFrame = df_out.copy()
def probe_generator(x, do_rt: bool = True) -> str:
    seq: str = x.probe_seq
    if not do_rt:
        seq = reverseComplement(seq)
    len_x: int = len(seq)
    right_len: int = (len_x // 2)
    return ''.join([seq[:right_len], # 5' is the right arm
                    RIGHT_LOOP,
                    x.barcode, 
                    T2S_SEQ, 
                    LEFT_LOOP, 
                    seq[right_len:] # 3' is the left arm
                   ]
    )

padlocks_list: List[str] = [probe_generator(df_padlocks.loc[i]) for i in range(len(df_padlocks)) ]
df_padlocks = df_padlocks.assign(padlock=padlocks_list)

In [11]:

df_padlocks[['symbol', 'probe_name', 'barcode', 'probe_length', 'padlock']]

,symbol,probe_name,barcode,probe_length,padlock
0,SLC17A8,65:723;,ACAGC,25,GTTCCTCATAGCTGCCCTTTCCTTTACAGCACTTCAGCTGCCCCGG...
1,SLC17A8,307:189;,ACAGC,25,GATAGAACTCAACCATGATTCCTTTACAGCACTTCAGCTGCCCCGG...
2,SLC17A8,589:220;,ACAGC,25,GAGTCCCAAAAAGAAGATTTCCTTTACAGCACTTCAGCTGCCCCGG...
3,SLC17A8,GE533132,ACAGC,30,AGAAGAAGGAATGGAAAGTTCCTTTACAGCACTTCAGCTGCCCCGG...
4,SLC17A8,A_14_P106429,ACAGC,60,TTCTCAACTATATCCTAATGTCTGAGAGGCTTCCTTTACAGCACTT...
5,SLC17A8,103:636;,ACAGC,25,ACTATATCCTAATGTCTGTTCCTTTACAGCACTTCAGCTGCCCCGG...
6,SLC17A8,674:405;,ACAGC,25,TCTGAGAGGCACTTCTGTTTCCTTTACAGCACTTCAGCTGCCCCGG...
7,SLC17A8,420:510;,ACAGC,25,TTAGAAACAGAAAGTATCTTCCTTTACAGCACTTCAGCTGCCCCGG...
8,SLC17A8,217:234;,ACAGC,25,GTATCCATACCTATTGCCTTCCTTTACAGCACTTCAGCTGCCCCGG...
9,SLC17A8,cg12083535,ACAGC,50,CGCTCAGTTGATAACATAGTTGATATTCCTTTACAGCACTTCAGCT...
